This notebook takes all of the preliminary SNeCO data and organizes it, preparing it for when we begin making intensity measurements in section 3.

To run this notebook we need '2.SNeCOMasterCatManualClean.txt' (see prepwork below). Once this prepwork is done, it is not needed to repeat unless we update the sample / redownload the OSC.

We also need "0.PHANGS_v1p6.csv" to pull other galaxy information from the survey. 
We also use "0.PHANGSMapFiles.txt" to pull the telescope orientations used for each galaxy.

      *PREPWORK* 
      
After looking at the galaxy images from #2. We find we need to remove one SN that isnt within the region covered by the PHANGS-ALMA footprint. We remove this (SN2003gd from NGC0628) and also the 6 candidate SNe, the one duplicate entry, and the LRN, and begin to perform some rudimentary stats in small tables.

Before we run this code we do the following prep:

We take the 2.SNeCOMasterCat.txt file and manually remove the following SNe:

Previously (& Permanently) Removed
AT2020nqq   (LRN) 
PTSS-19clju (Duplicate)

Recently commented out
AT2018eoq   (Candidate)
AT2019pck   (Candidate)
AT2019npi   (Candidate)
AT2019npd   (Candidate)
AT2020hol   (Candidate)
AT2020juh   (Candidate)
AT2020cwh   (Candidate)
SN2003gd    (Out of Map ngc0628)
SN2005af    (Out of Map ngc4945)
ngc5236,PS  (Not in Map & Also not good data on this one)

*also shortened type IIn, IIb to II and type Ia-91bg to Ia, etc.

Then we save this as 2.SNeCOMasterCatManualClean.txt

We turn this file into a dataframe and use to make basic tables of stats.
**********************
    
    *MORE PREPWORK* Changed 'name' column to 'galaxy' & 'eso097-013' to 'circinus' in 0.PHANGS_v1p6.csv file
    

In [1]:
# IMPORT PACKAGES

import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.io import ascii
import astropy
from astropy.io import fits

In [2]:
# Turn file into dataStructure and clean

dataFile      = '../Data/2.SNeCOMasterCatManualClean.txt'
galaxy, SNname, SNtype = np.genfromtxt(dataFile,  dtype=str, unpack=True, delimiter = ',', usecols=(0,1,2))
ra, dec, xVal, yVal, co_21_int, co_21_err, co_21_ew, beamsize  = np.genfromtxt(dataFile,  dtype=float, unpack=True, delimiter = ',', usecols=(2,3,4,5,7,8,9,10))

# remove spaces and " " from SN data
galaxy = [x.strip() for x in galaxy]
#SNname = [x[2:-2] for x in SNname]
SNtype = [x.strip() for x in SNtype]

# build table 
data = Table({'galaxy'       : galaxy,
              'SN_name'      : SNname,
              'SN_type'      : SNtype,
              'SN_ra'        : ra,
              'SN_dec'       : dec,
              'SN_xCoord'    : xVal,
              'SN_yCoord'    : yVal,
              'SN_co21int'   : co_21_int,
              'SN_co21err'   : co_21_err,
              'SN_co21ew'    : co_21_ew,
              'map_beamsize' : beamsize,
             },
             names=['galaxy', 'SN_name','SN_type','SN_ra','SN_dec','SN_xCoord', 'SN_yCoord', 'SN_co21int', 'SN_co21err', "SN_co21ew", 'map_beamsize'])


In [3]:
# Pull Galaxy data from PHANGS survey

PHANGS_data = '../Data/0.PHANGS_v1p6.csv'
PHANGSTable = Table.read(PHANGS_data, format='csv')

PHANGS_clean = PHANGSTable['galaxy', 'orient_ra', 'orient_dec', 'dist', 'orient_posang', 'orient_incl', 'size_r25']

In [4]:
for i in range(len(PHANGSTable)):
    print(PHANGSTable['galaxy', 'survey_alma_status'][i])


 galaxy  survey_alma_status
-------- ------------------
circinus           released
galaxy survey_alma_status
------ ------------------
ic0010 cannot_be_observed
galaxy survey_alma_status
------ ------------------
ic0342 cannot_be_observed
galaxy survey_alma_status
------ ------------------
ic1954           released
galaxy survey_alma_status
------ ------------------
ic1993      not_in_survey
galaxy survey_alma_status
------ ------------------
ic5273           released
galaxy survey_alma_status
------ ------------------
ic5332           released
 galaxy survey_alma_status
------- ------------------
ngc0224 cannot_be_observed
 galaxy survey_alma_status
------- ------------------
ngc0247           released
 galaxy survey_alma_status
------- ------------------
ngc0253           released
 galaxy survey_alma_status
------- ------------------
ngc0278 cannot_be_observed
 galaxy survey_alma_status
------- ------------------
ngc0300           released
 galaxy survey_alma_status
------- --------

In [5]:
SN_plus_gal_data = join(data, PHANGS_clean, keys='galaxy')

# SN_plus_gal_data.colnames

In [6]:
# Pull Telescope Orientation data from PHANGSMapFiles.csv and join to table

telOrientFile = '../Data/0.PHANGSMapFiles.csv'
telOrientTable = Table.read(telOrientFile, format='csv')

SN_plus_gal_plus_telOrient_data = join(SN_plus_gal_data,telOrientTable, keys='galaxy')


In [7]:
SN_plus_gal_plus_telOrient_data

galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient
str8,str12,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9
circinus,SN1996cr,II,nan,213.2918,-65.3457,86.56761411031889,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp
ngc0253,SN1940E,I,nan,11.8783,-25.2934,296.9010534051223,219.30367,1.465811,45.862267,0.00232543637022,11.887966,-25.288443,3.7,52.48,75.0,803.7504,7m+tp
ngc0300,SN2019qyl,II,nan,13.7399,-37.6444,124.97990204934794,-0.41793606,0.4800788,nan,0.002271061982879,13.723024,-37.684475,2.09,114.3,39.8,582.26587,7m+tp
ngc0628,SN2019krl,II,nan,24.2068,15.7795,227.48504797816918,0.2809551,1.2958647,nan,0.0003114595189268,24.173855,15.783643,9.84,20.7,8.9,296.56592,12m+7m+tp
ngc0628,SN2013ej,II,nan,24.2007,15.7586,333.46022572413716,5.7713037,1.9949342,nan,0.0003114595189268,24.173855,15.783643,9.84,20.7,8.9,296.56592,12m+7m+tp
ngc1068,SN2018ivc,II,nan,40.672,-0.0088,41.400000046852824,127.77749,0.6721418,32.0753,0.002414131938401,40.66973,-0.0132779,13.97,72.65,34.7,183.28261,7m+tp
ngc1087,SN1995V,II,nan,41.6115,-0.4988,161.4198245203906,11.513839,1.1399242,5.175495,0.0004451676280474,41.60492,-0.498717,15.85,359.1,42.9,89.14999,12m+7m+tp
ngc1097,SN1999eu,II,nan,41.5866,-30.3184,245.2470996170838,0.0,0.7541714,nan,0.0004711325009833,41.578957,-30.274675,13.58,122.4,48.6,317.0452,12m+7m+tp
ngc1097,SN1992bd,II,nan,41.5792,-30.2756,291.5099030167368,296.15988,2.0283754,45.82589,0.0004711325009833,41.578957,-30.274675,13.58,122.4,48.6,317.0452,12m+7m+tp


In [8]:
ascii.write(SN_plus_gal_plus_telOrient_data, '../Data/3.SNe+GalData.csv', format='csv', fast_writer=False, overwrite=True)  